In [35]:
import pandas as pd
import sqlite3

# Read the SQL file
with open('restaurants.sql', 'r') as f:
    sql_script = f.read()

# Connect to the database and execute the SQL script
conn = sqlite3.connect('restaurants.db')
cursor = conn.cursor()

# Drop table if it already exists to prevent OperationalError
cursor.execute('DROP TABLE IF EXISTS restaurants;')
cursor.executescript(sql_script)
conn.commit()

orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
conn.close()

merged = orders.merge(users, on='user_id', how='left')
merged = merged.merge(restaurants, on='restaurant_id', how='left')
merged.to_csv('final-food_delivery_dataset.csv', index=False)

In [34]:
merged = merged.drop(columns=['restaurant_name_y'])
merged = merged.rename(columns={'restaurant_name_x': 'restaurant_name'})

display(merged.head())
merged.info()

KeyError: "['restaurant_name_y'] not found in axis"

In [5]:
display(merged.head())

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [8]:
# Filter the dataset for gold members
gold_members = merged[merged['membership'] == 'Gold']

# Group by city and sum the total amount
city_revenue = gold_members.groupby('city')['total_amount'].sum()

# Find the city with the highest revenue
highest_revenue_city = city_revenue.idxmax()
highest_revenue_amount = city_revenue.max()

print(f"The city with the highest total revenue from gold members is {highest_revenue_city} with a total revenue of {highest_revenue_amount}.")


The city with the highest total revenue from gold members is Chennai with a total revenue of 1080909.79.


In [9]:
# Group by cuisine and calculate the average order value
cuisine_avg_order = merged.groupby('cuisine')['total_amount'].mean()

# Find the cuisine with the highest average order value
highest_avg_cuisine = cuisine_avg_order.idxmax()
highest_avg_value = cuisine_avg_order.max()

print(f"The cuisine with the highest average order value is {highest_avg_cuisine} with an average order value of {highest_avg_value:.2f}.")



The cuisine with the highest average order value is Mexican with an average order value of 808.02.


In [10]:
# Group by user_id and sum their total order amounts
user_totals = merged.groupby('user_id')['total_amount'].sum()

# Filter users whose total order amount is greater than 1000
high_value_users = user_totals[user_totals > 1000]

# Count the number of distinct users
num_distinct_users = high_value_users.count()

print(f"Number of distinct users with total orders greater than 1000 is {num_distinct_users}.")


Number of distinct users with total orders greater than 1000 is 2544.


In [11]:
# Group by restaurant rating and sum the total revenue
rating_revenue = merged.groupby('rating')['total_amount'].sum()

# Find the rating range with the highest total revenue
highest_revenue_rating = rating_revenue.idxmax()
highest_revenue_amount = rating_revenue.max()

print(f"The restaurant rating range with the highest total revenue is {highest_revenue_rating} with a total revenue of {highest_revenue_amount:.2f}.")


The restaurant rating range with the highest total revenue is 4.8 with a total revenue of 657707.71.


In [12]:
# Filter the dataset for gold members
gold_members = merged[merged['membership'] == 'Gold']

# Group by city and calculate the average order value
city_avg_order = gold_members.groupby('city')['total_amount'].mean()

# Find the city with the highest average order value
highest_avg_city = city_avg_order.idxmax()
highest_avg_value = city_avg_order.max()

print(f"The city with the highest average order value among gold members is {highest_avg_city} with an average order value of {highest_avg_value:.2f}.")


The city with the highest average order value among gold members is Chennai with an average order value of 808.46.


In [13]:
# Group by cuisine and restaurant to count distinct restaurants
cuisine_restaurant_counts = merged.groupby(['cuisine', 'restaurant_id'])['restaurant_id'].nunique().groupby('cuisine').count()

# Calculate total revenue for each cuisine
cuisine_revenue = merged.groupby('cuisine')['total_amount'].sum()

# Combine the data into a single DataFrame
cuisine_data = pd.DataFrame({
    'distinct_restaurants': cuisine_restaurant_counts,
    'total_revenue': cuisine_revenue
})

# Sort by distinct restaurants ascending and then by total revenue descending
sorted_cuisine = cuisine_data.sort_values(by=['distinct_restaurants', 'total_revenue'], ascending=[True, False])

# Get the cuisine with the lowest number of distinct restaurants but significant revenue
target_cuisine = sorted_cuisine.head(1)

print(f"The cuisine with the lowest number of distinct restaurants but significant revenue is {target_cuisine.index[0]} with {target_cuisine['distinct_restaurants'].values[0]} distinct restaurants and a total revenue of {target_cuisine['total_revenue'].values[0]:.2f}.")


The cuisine with the lowest number of distinct restaurants but significant revenue is Chinese with 120 distinct restaurants and a total revenue of 1930504.65.


In [14]:
# Total revenue from all orders
total_revenue = merged['total_amount'].sum()

# Total revenue from gold members
gold_revenue = merged[merged['membership'] == 'Gold']['total_amount'].sum()

# Calculate the percentage and round to the nearest integer
percent_gold_members = round((gold_revenue / total_revenue) * 100)

print(f"Percentage of total orders placed by gold members: {percent_gold_members}%")


Percentage of total orders placed by gold members: 50%


In [23]:
import pandas as pd

# Assume 'merged' is your combined DataFrame

# Group by restaurant and calculate total orders and average order value
restaurant_stats = merged.groupby('restaurant_id').agg(
    total_orders=('order_id', 'count'),
    average_order_value=('total_amount', 'mean')
)

# Get restaurant names
restaurant_names = merged[['restaurant_id', 'restaurant_name']].drop_duplicates().set_index('restaurant_id')

# Combine the stats with the restaurant names
restaurant_stats = restaurant_stats.join(restaurant_names)

# Filter restaurants with less than 20 total orders
filtered_restaurants = restaurant_stats[restaurant_stats['total_orders'] < 20]

# Find the restaurant with the highest average order value in the filtered set
highest_avg_restaurant = filtered_restaurants.sort_values(by='average_order_value', ascending=False).head(1)

# Extract the restaurant ID, name, and average order value
restaurant_id = highest_avg_restaurant.index[0]
restaurant_name = highest_avg_restaurant['restaurant_name'].values[0]
avg_order_value = highest_avg_restaurant['average_order_value'].values[0]

print(f"The restaurant with the highest average order value but less than 20 total orders is {restaurant_name} (ID: {restaurant_id}) with an average order value of {avg_order_value:.2f}.")


The restaurant with the highest average order value but less than 20 total orders is Restaurant_294 (ID: 294) with an average order value of 1040.22.


In [28]:
display(merged.head(200))

,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name,cuisine,rating
0,1,2508,450,18-02-2023,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...
195,196,789,254,25-07-2023,967.60,User_789,Pune,Regular,Restaurant_254,Indian,4.7
196,197,996,58,03-03-2023,619.68,User_996,Chennai,Gold,Restaurant_58,Indian,4.4
197,198,767,243,05-04-2023,1389.57,User_767,Hyderabad,Gold,Restaurant_243,Chinese,4.8
198,199,2543,94,06-10-2023,844.75,User_2543,Pune,Regular,Restaurant_94,Chinese,4.5


In [29]:
# Filter the merged DataFrame for the specific restaurant ID
restaurant_id = 294
restaurant_data = merged[merged['restaurant_id'] == restaurant_id]

# Check if the restaurant data is found
if not restaurant_data.empty:
    print(restaurant_data)
else:
    print(f"Restaurant with ID {restaurant_id} does not exist in the data.")


      order_id  user_id  restaurant_id  order_date  total_amount       name  \
1407      1408     2895            294  22-08-2023        817.71  User_2895   
1643      1644     1363            294  29-09-2023       1327.91  User_1363   
2426      2427     2944            294  30-08-2023        629.97  User_2944   
3174      3175      315            294  20-11-2023       1004.28   User_315   
3243      3244     2100            294  08-03-2023       1189.93  User_2100   
4007      4008     2137            294  13-02-2023       1082.37  User_2137   
4999      5000     1478            294  24-10-2023        943.66  User_1478   
5157      5158      701            294  08-07-2023       1290.51   User_701   
5718      5719     1223            294  13-10-2023       1400.13  User_1223   
5851      5852     2089            294  18-11-2023       1175.40  User_2089   
6155      6156      545            294  01-01-2024       1070.86   User_545   
8840      8841     2381            294  04-12-2023  

In [30]:
# Filter the merged DataFrame for the specific restaurant ID
restaurant_id = 294
restaurant_data = merged[merged['restaurant_id'] == restaurant_id]

# Check if the restaurant data is found
if not restaurant_data.empty:
    print(restaurant_data)
else:
    print(f"Restaurant with ID {restaurant_id} does not exist in the data.")


      order_id  user_id  restaurant_id  order_date  total_amount       name  \
1407      1408     2895            294  22-08-2023        817.71  User_2895   
1643      1644     1363            294  29-09-2023       1327.91  User_1363   
2426      2427     2944            294  30-08-2023        629.97  User_2944   
3174      3175      315            294  20-11-2023       1004.28   User_315   
3243      3244     2100            294  08-03-2023       1189.93  User_2100   
4007      4008     2137            294  13-02-2023       1082.37  User_2137   
4999      5000     1478            294  24-10-2023        943.66  User_1478   
5157      5158      701            294  08-07-2023       1290.51   User_701   
5718      5719     1223            294  13-10-2023       1400.13  User_1223   
5851      5852     2089            294  18-11-2023       1175.40  User_2089   
6155      6156      545            294  01-01-2024       1070.86   User_545   
8840      8841     2381            294  04-12-2023  

In [49]:
restaurant_name = restaurant_name_294
print(f"The value of restaurant_name is: {restaurant_name}")

The value of restaurant_name is: Hotel Dhaba Multicuisine


In [47]:
restaurant_name_294 = merged[merged['restaurant_id'] == 294]['restaurant_name'].iloc[0]
print(f"The name of the restaurant with ID 294 is: {restaurant_name_294}")

The name of the restaurant with ID 294 is: Hotel Dhaba Multicuisine


In [39]:
# Filter the dataset for gold members
gold_members = merged[merged['membership'] == 'Gold']

# Group by city and sum the total amount
city_revenue = gold_members.groupby('city')['total_amount'].sum()

# Find the city with the highest revenue
highest_revenue_city = city_revenue.idxmax()
highest_revenue_amount = city_revenue.max()

print(f"The city with the highest total revenue from gold members is {highest_revenue_city} with a total revenue of {highest_revenue_amount:.2f}.")

The city with the highest total revenue from gold members is Chennai with a total revenue of 1080909.79.


In [40]:
# Group by cuisine and calculate the average order value
cuisine_avg_order = merged.groupby('cuisine')['total_amount'].mean()

# Find the cuisine with the highest average order value
highest_avg_cuisine = cuisine_avg_order.idxmax()
highest_avg_value = cuisine_avg_order.max()

print(f"The cuisine with the highest average order value is {highest_avg_cuisine} with an average order value of {highest_avg_value:.2f}.")

The cuisine with the highest average order value is Mexican with an average order value of 808.02.


In [41]:
# Group by user_id and sum their total order amounts
user_totals = merged.groupby('user_id')['total_amount'].sum()

# Filter users whose total order amount is greater than 1000
high_value_users = user_totals[user_totals > 1000]

# Count the number of distinct users
num_distinct_users = high_value_users.count()

print(f"Number of distinct users with total orders greater than 1000 is {num_distinct_users}.")

Number of distinct users with total orders greater than 1000 is 2544.


In [42]:
# Group by restaurant rating and sum the total revenue
rating_revenue = merged.groupby('rating')['total_amount'].sum()

# Find the rating range with the highest total revenue
highest_revenue_rating = rating_revenue.idxmax()
highest_revenue_amount = rating_revenue.max()

print(f"The restaurant rating range with the highest total revenue is {highest_revenue_rating} with a total revenue of {highest_revenue_amount:.2f}.")

The restaurant rating range with the highest total revenue is 4.8 with a total revenue of 657707.71.


In [43]:
# Filter the dataset for gold members
gold_members = merged[merged['membership'] == 'Gold']

# Group by city and calculate the average order value
city_avg_order = gold_members.groupby('city')['total_amount'].mean()

# Find the city with the highest average order value
highest_avg_city = city_avg_order.idxmax()
highest_avg_value = city_avg_order.max()

print(f"The city with the highest average order value among gold members is {highest_avg_city} with an average order value of {highest_avg_value:.2f}.")

The city with the highest average order value among gold members is Chennai with an average order value of 808.46.


In [44]:
# Group by cuisine and restaurant to count distinct restaurants
cuisine_restaurant_counts = merged.groupby(['cuisine', 'restaurant_id'])['restaurant_id'].nunique().groupby('cuisine').count()

# Calculate total revenue for each cuisine
cuisine_revenue = merged.groupby('cuisine')['total_amount'].sum()

# Combine the data into a single DataFrame
cuisine_data = pd.DataFrame({
    'distinct_restaurants': cuisine_restaurant_counts,
    'total_revenue': cuisine_revenue
})

# Sort by distinct restaurants ascending and then by total revenue descending
sorted_cuisine = cuisine_data.sort_values(by=['distinct_restaurants', 'total_revenue'], ascending=[True, False])

# Get the cuisine with the lowest number of distinct restaurants but significant revenue
target_cuisine = sorted_cuisine.head(1)

print(f"The cuisine with the lowest number of distinct restaurants but significant revenue is {target_cuisine.index[0]} with {target_cuisine['distinct_restaurants'].values[0]} distinct restaurants and a total revenue of {target_cuisine['total_revenue'].values[0]:.2f}.")

The cuisine with the lowest number of distinct restaurants but significant revenue is Chinese with 120 distinct restaurants and a total revenue of 1930504.65.


In [45]:
# Total revenue from all orders
total_revenue = merged['total_amount'].sum()

# Total revenue from gold members
gold_revenue = merged[merged['membership'] == 'Gold']['total_amount'].sum()

# Calculate the percentage and round to the nearest integer
percent_gold_members = round((gold_revenue / total_revenue) * 100)

print(f"Percentage of total orders placed by gold members: {percent_gold_members}%")

Percentage of total orders placed by gold members: 50%


In [46]:
import pandas as pd

# Assume 'merged' is your combined DataFrame

# Group by restaurant and calculate total orders and average order value
restaurant_stats = merged.groupby('restaurant_id').agg(
    total_orders=('order_id', 'count'),
    average_order_value=('total_amount', 'mean')
)

# Get restaurant names
restaurant_names = merged[['restaurant_id', 'restaurant_name']].drop_duplicates().set_index('restaurant_id')

# Combine the stats with the restaurant names
restaurant_stats = restaurant_stats.join(restaurant_names)

# Filter restaurants with less than 20 total orders
filtered_restaurants = restaurant_stats[restaurant_stats['total_orders'] < 20]

# Find the restaurant with the highest average order value in the filtered set
highest_avg_restaurant = filtered_restaurants.sort_values(by='average_order_value', ascending=False).head(1)

# Extract the restaurant ID, name, and average order value
restaurant_id = highest_avg_restaurant.index[0]
restaurant_name = highest_avg_restaurant['restaurant_name'].values[0]
avg_order_value = highest_avg_restaurant['average_order_value'].values[0]

print(f"The restaurant with the highest average order value but less than 20 total orders is {restaurant_name} (ID: {restaurant_id}) with an average order value of {avg_order_value:.2f}.")

The restaurant with the highest average order value but less than 20 total orders is Hotel Dhaba Multicuisine (ID: 294) with an average order value of 1040.22.


In [38]:
# The content of this cell was duplicated from cell 'duABmYT26Or4'. It has been removed for notebook clarity.

In [37]:
merged = merged.drop(columns=['restaurant_name_y'])
merged = merged.rename(columns={'restaurant_name_x': 'restaurant_name'})

display(merged.head())
merged.info()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Italian,3.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
 6   name             10000 non-null  object 
 7   city             10000 non-null  object 
 8   membership       10000 non-null  object 
 9   cuisine          10000 non-null  object 
 10  rating           10000 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 859.5+ KB


In [31]:
restaurant_294_data = merged[merged['restaurant_id'] == 294]
display(restaurant_294_data)

,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name,cuisine,rating
1407,1408,2895,294,22-08-2023,817.71,User_2895,Chennai,Gold,Restaurant_294,Italian,3.1
1643,1644,1363,294,29-09-2023,1327.91,User_1363,Bangalore,Gold,Restaurant_294,Italian,3.1
2426,2427,2944,294,30-08-2023,629.97,User_2944,Hyderabad,Gold,Restaurant_294,Italian,3.1
3174,3175,315,294,20-11-2023,1004.28,User_315,Chennai,Gold,Restaurant_294,Italian,3.1
3243,3244,2100,294,08-03-2023,1189.93,User_2100,Hyderabad,Regular,Restaurant_294,Italian,3.1
4007,4008,2137,294,13-02-2023,1082.37,User_2137,Pune,Gold,Restaurant_294,Italian,3.1
4999,5000,1478,294,24-10-2023,943.66,User_1478,Pune,Gold,Restaurant_294,Italian,3.1
5157,5158,701,294,08-07-2023,1290.51,User_701,Hyderabad,Regular,Restaurant_294,Italian,3.1
5718,5719,1223,294,13-10-2023,1400.13,User_1223,Chennai,Regular,Restaurant_294,Italian,3.1
5851,5852,2089,294,18-11-2023,1175.40,User_2089,Hyderabad,Gold,Restaurant_294,Italian,3.1


In [51]:
revenue_combination = merged.groupby(['restaurant_name', 'country'])['total_amount'].sum()
best_combo = revenue_combination.idxmax()
best_revenue = revenue_combination.max()

print(best_combo, best_revenue)


KeyError: 'country'

In [52]:
# Filter for Gold members
gold_members_df = merged[merged['membership'] == 'Gold']

# Group by cuisine and sum total_amount for gold members
gold_cuisine_revenue = gold_members_df.groupby('cuisine')['total_amount'].sum()

# Get the revenue for Indian cuisine
indian_cuisine_gold_revenue = gold_cuisine_revenue.get('Indian', 0)

print(f"Total revenue from Gold members for Indian cuisine: {indian_cuisine_gold_revenue:.2f}")

Total revenue from Gold members for Indian cuisine: 979312.31


In [53]:
# Group by membership and cuisine and sum the total amount
revenue_by_membership_cuisine = merged.groupby(['membership', 'cuisine'])['total_amount'].sum()

# Find the combination with the highest revenue
highest_revenue_combination = revenue_by_membership_cuisine.idxmax()
highest_revenue_amount = revenue_by_membership_cuisine.max()

print(f"The combination that generated the highest revenue is {highest_revenue_combination} with a total revenue of {highest_revenue_amount:.2f}.")

The combination that generated the highest revenue is ('Regular', 'Mexican') with a total revenue of 1072943.30.


In [55]:
merged['order_date'] = pd.to_datetime(merged['order_date'], dayfirst=True)
merged['month'] = merged['order_date'].dt.month

merged['quarter_label'] = pd.cut(
    merged['month'],
    bins=[0,3,6,9,12],
    labels=["January to March","April to June","July to September","October to December"]
)

quarter_revenue = merged.groupby('quarter_label')['total_amount'].sum()
best_quarter = quarter_revenue.idxmax()
best_revenue = quarter_revenue.max()

print(best_quarter, best_revenue)


July to September 2037385.1


/tmp/ipython-input-2831413972.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  quarter_revenue = merged.groupby('quarter_label')['total_amount'].sum()


In [56]:
gold_orders_count = merged[merged['membership'] == 'Gold']['order_id'].count()
print(gold_orders_count)


4987


In [57]:
hyderabad_revenue = merged[merged['city'] == 'Hyderabad']['total_amount'].sum()
hyderabad_revenue_rounded = round(hyderabad_revenue)
print(hyderabad_revenue_rounded)


1889367


In [58]:
distinct_users = merged['user_id'].nunique()
print(distinct_users)


2883


In [59]:
gold_avg_order = merged[merged['membership'] == 'Gold']['total_amount'].mean()
print(round(gold_avg_order, 2))


797.15


In [60]:
high_rating_orders = merged[merged['rating'] >= 4.5]['order_id'].count()
print(high_rating_orders)


3374


In [61]:
gold = merged[merged['membership'] == 'Gold']

city_revenue = gold.groupby('city')['total_amount'].sum()
top_city = city_revenue.idxmax()

orders_in_top_city = gold[gold['city'] == top_city]['order_id'].count()
print(orders_in_top_city)


1337


In [62]:
total_rows = merged.shape[0]
print(total_rows)


10000
